## Setup Spark and SQL Contexts

In [9]:
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext

sparkContext = SparkContext.getOrCreate()
sqlContext = SQLContext(sparkContext)

sqlContext

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.io.FileNotFoundException: Added file file:/root/pipeline/myapps/pmml/spark/1.6.1/lib/jpmml.py does not exist.
	at org.apache.spark.SparkContext.addFile(SparkContext.scala:1364)
	at org.apache.spark.SparkContext.addFile(SparkContext.scala:1340)
	at org.apache.spark.SparkContext$$anonfun$15.apply(SparkContext.scala:491)
	at org.apache.spark.SparkContext$$anonfun$15.apply(SparkContext.scala:491)
	at scala.collection.immutable.List.foreach(List.scala:318)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:491)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:59)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.lang.refle

In [ ]:
sys.path

In [ ]:
!export

## Load Dataset into Spark Cluster

In [ ]:
data = sqlContext.read.format("csv").option("inferSchema", "true").option("header", "true").load("/root/datasets/R/census.csv")

data.collect()

## Build Decision Tree (Regression) with Spark ML Pipeline

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import RFormula
from pyspark.ml.classification import DecisionTreeClassifier

formula = RFormula(formula = "income ~ .")
classifier = DecisionTreeClassifier()
pipeline = Pipeline(stages = [formula, classifier])
pipelineModel = pipeline.fit(data)

pipelineModel

## Convert Spark ML Model and Pipeline to PMML

In [ ]:
from jpmml import toPMMLBytes

pmmlBytes = toPMMLBytes(sparkContext, data, pipelineModel)

str(pmmlBytes)

In [ ]:
import urllib.request

update_url = 'http://prediction.datasticks.com:9040/update-pmml/census'

update_headers = {}
update_headers['Content-type'] = 'application/json'

req = urllib.request.Request(update_url, headers=update_headers, data=pmmlBytes)
resp = urllib.request.urlopen(req)

print(resp.status)

In [ ]:
import urllib.parse
import json

evaluate_url = 'http://prediction.datasticks.com:9040/evaluate-pmml/census'

evaluate_headers = {}
evaluate_headers['Content-type'] = 'application/json'
input_params = '{"age":39,"workclass":"State-gov","education":"Bachelors","education_num":13,"marital_status":"Never-married","occupation":"Adm-clerical","relationship":"Not-in-family","race":"White","sex":"Male","capital_gain":2174,"capital_loss":0,"hours_per_week":40,"native_country":"United-States"}' 
#encoded_input_params = urllib.parse.urlencode(input_params)
encoded_input_params = input_params.encode('utf-8')
#encoded_input_params = data.encode('utf-8') # data should be bytes

req = urllib.request.Request(evaluate_url, headers=evaluate_headers, data=encoded_input_params)
resp = urllib.request.urlopen(req)

print(resp.read())